Import libraries

In [55]:
from collections import defaultdict, deque

Import datasets

In [56]:
test_order = """47|53
97|13
97|61
97|47
75|29
61|13
75|53
29|13
97|29
53|29
61|53
97|53
61|29
47|13
75|47
97|75
47|61
75|61
47|29
75|13
53|13"""

test_pages = """75,47,61,53,29
97,61,53,29,13
75,29,13
75,97,47,61,53
61,13,29
97,13,75,29,47"""

In [57]:
with open('day5input.txt', 'r') as f:
    data=f.read()

Define functions

In [58]:
def make_sequences(list_of_pages):
    sequences = []
    for pages in list_of_pages.splitlines():
        sequences.append(pages.split(","))
    return sequences

def create_order(pairs_list):
    graph = defaultdict(list)
    in_degree = defaultdict(int)
    
    for pair in pairs_list.splitlines():
        a, b = pair.split("|")
        graph[a].append(b)
        in_degree[b] += 1
        if a not in in_degree:
            in_degree[a] = 0
            
    return graph, in_degree


def topological_sort(graph, in_degree):
    queue = deque([node for node in in_degree if in_degree[node] == 0])
    topological_order = []
    #print(queue)
    while queue:
        node = queue.popleft()
        topological_order.append(node)
        for neighbor in graph[node]:
            in_degree[neighbor] -= 1
            if in_degree[neighbor] == 0:
                queue.append(neighbor)
    
    return topological_order


def create_subgraph(sequence, global_graph, global_in_degree):
    subgraph = defaultdict(list)
    sub_in_degree = defaultdict(int)
    sequence_set = set(sequence)
    
    for node in sequence_set:
        if node in global_graph:
            for neighbor in global_graph[node]:
                if neighbor in sequence_set:
                    subgraph[node].append(neighbor)
                    sub_in_degree[neighbor] += 1
            if node not in sub_in_degree:
                sub_in_degree[node] = 0

    return subgraph, sub_in_degree

def check_sequence(sequence, valid_order,printing=True):
    #just slap enumerate on everything and pretend I'm a programmer
    #print(valid_order)
    pos = {node: index for index, node in enumerate(valid_order)}
    #print(pos)
    for i in range(len(sequence) - 1):
        if sequence[i] not in pos:
            continue
        if pos[sequence[i]] > pos[sequence[i + 1]]:
            if printing: print(f"😫: {sequence}")
            return False,[]
    
    if printing: print(f"👌: {sequence} middle : {sequence[len(sequence)//2]}")
    return True,sequence[len(sequence)//2]

def sort_sequence(sequence, valid_order):
    #print(valid_order)
    pos = {node: index for index, node in enumerate(valid_order)}
    #print(pos)
    sorted_sequence = sorted(sequence, key=lambda x: pos.get(x, float('inf')))
    
    #print(f"og sequence: {sequence}")
    #print(f"sorted sequence: {sorted_sequence}")
    return sorted_sequence


Run example

In [59]:
graph, connections = create_order(test_order)
valid_order = topological_sort(graph, connections)
total=0
sequences = make_sequences(test_pages)
for sequence in sequences:
    valid,middle=check_sequence(sequence, valid_order)
    if valid:
        total+=int(middle)
print(f"Test : {total} == 143")

👌: ['75', '47', '61', '53', '29'] middle : 61
👌: ['97', '61', '53', '29', '13'] middle : 53
👌: ['75', '29', '13'] middle : 29
😫: ['75', '97', '47', '61', '53']
😫: ['61', '13', '29']
😫: ['97', '13', '75', '29', '47']
Test : 143 == 143


Run Part 1

In [60]:
ordering,checking=data.split("\n\n")
graph, connections = create_order(ordering)
# for node, edges in graph.items():
#     print(f"Node {node} points to: {edges}")
#print(graph)
#print(connections)
valid_order = topological_sort(graph, connections)
#print("toporder :", valid_order)
total=0
sequences = make_sequences(checking)
for sequence in sequences:
    
    subgraph, sub_in_degree = create_subgraph(sequence, graph, connections)
    valid_order = topological_sort(subgraph, sub_in_degree)

    
    valid, middle = check_sequence(sequence, valid_order)
    if valid:
        total += int(middle)
    
print(f"Real Input Total: {total}")

😫: ['42', '54', '21', '36', '22', '33', '13', '29', '35']
😫: ['83', '67', '22', '14', '78', '99', '18', '92', '15', '77', '52', '68', '82', '55', '21', '61', '85', '91', '51', '64', '72', '24', '88']
👌: ['85', '67', '64', '82', '61', '63', '44', '71', '38', '97', '43', '54', '42', '32', '29', '35', '65', '37', '13', '48', '33', '23', '36'] middle : 54
👌: ['96', '81', '21', '14', '52', '99', '88', '55', '51', '15', '77', '68', '72', '18', '85'] middle : 55
😫: ['63', '52', '77', '85', '91', '83', '22', '61', '14', '64', '82', '68', '51', '24', '55']
👌: ['63', '97', '76', '32', '29', '35', '13', '48', '33', '36', '25', '56', '96', '89', '17'] middle : 48
😫: ['15', '99', '51', '88', '21', '83', '72', '56', '81', '18', '92', '52', '55', '17', '89', '96', '87']
😫: ['92', '22', '17', '96', '78', '87', '72', '14', '24', '55', '81', '91', '52', '83', '68', '88', '18', '99', '56']
😫: ['87', '18', '61', '67', '64', '76', '72', '85', '97', '65', '91', '68', '32', '44', '29', '43', '37']
😫: ['87', 

Part 2

In [61]:
ordering,checking=data.split("\n\n")
graph, connections = create_order(ordering)
# for node, edges in graph.items():
#     print(f"Node {node} points to: {edges}")
#print(graph)
#print(connections)
valid_order = topological_sort(graph, connections)
#print("toporder :", valid_order)
total=0
total_missprints=0
sequences = make_sequences(checking)

for sequence in sequences:
    subgraph, sub_in_degree = create_subgraph(sequence, graph, connections)
    valid_order = topological_sort(subgraph, sub_in_degree)
    
    valid, middle = check_sequence(sequence, valid_order,False)
    if valid:
        total += int(middle)
    else:
        p2_sorted_sequence=sort_sequence(sequence,valid_order)
        total_missprints+=int(p2_sorted_sequence[len(p2_sorted_sequence)//2])
        #print(p2_sorted_sequence)
        

print(f"missprints Total: {total_missprints}")

missprints Total: 6311
